In [1]:
import tensorflow as tf

# 激活函数

In [1]:

tf.nn.relu
tf.sigmoid
tf.tanh

<function tensorflow.python.ops.math_ops.tanh>

In [ ]:
a=tf.nn.relu(tf.matmul(x,w1)+biases1)
y=tf.nn.relu(tf.matmul(a,w2)+biases2)

In [ ]:
keep_prob=tf.placeholder(tf.float32)#训练时小于1，预测时等于1，所以还是用placeholder
hidden1_drop=tf.nn.dropout(hidden1,keep_prob)

## CNN相关

In [ ]:
tf.nn.conv2d(x,kernel,strides=[1,1,1,1],padding='SAME')
#strides[0,1]数表示卷积核的尺寸，
#stride[2]表示channel数量，灰度图片是1，RGB图是3
#stride[3]表示卷积核的数量
#padding='SAME'表示边界加上padding，使得卷积的输入和输出保持同样的尺寸

tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')#横竖步长为2，

In [ ]:
tf.nn.lrn(pool1,4,bias=1,alpha=0.001/9.0,beta=0.75)
#LRN模仿生物的侧抑制机制，对局部神经元创建竞争环境，使其中响应大的值相对更大，并抑制其它反馈小的神经元。从而增强泛化能力
#可以用于Pooling之后，也可以用于conv之后、Pooling之前
#适用于ReLu这种没有上界的激活函数，不适合Sigmoid这种有固定边界，或者能抑制过大值得激活函数

# 损失函数
#### 1.交叉熵
$H(p,q)=-\sum p(x)\log q(x)$

In [ ]:
cross_entropy=-tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
# y_代表正确结果，y代表预测结果
# tf.clip_by_value可以把张量中的数值限制在某个范围内,这里为了防止log0报错
# reduce_mean:大概是求均值？

In [ ]:
# TensorFlow对softmax+crossentropy进行了封装
cross_entropy=tf.nn.softmax_cross_entropy_with_logits(y,y_)

#### 2.MSE
$MSE(y,y')=\dfrac{\sum (y-y')^2}{n}$

In [ ]:
mse=tf.reduce_mean(tf.square(y_-y))

#### 3.自定义
例如，预测销量时，多预测一个损失1元，少预测1个损失10元。  
$Loss(y,y')=\sum f(y_i,y_i')$,
$f(x,y)=\left\{\begin{array}{ccc}a(x-y)&x>y\\
b(y-x)&x\leq y\end{array}\right.$

In [ ]:
loss=tf.reduce_sum(tf.select(tf.greater(v1,v2),(v1-v2)*a,(v2-v1)*b))


# 学习率
decayed_learning_rate=learning_rate * decay_rate ** (global_step/decay_steps)
```
tf.train.exponential_decay# 指数下降法减小学习率
learning_rate=tf.train.exponential_decay(0.01,global_step=10000,decay_steps=100,decay_rate=0.96,staircase=True)
```
staircase=True时，(global_step/decay_steps)会转化成整数，导致学习率阶梯下降

In [2]:
import tensorflow as tf
TRAINING_STEPS = 100
global_step = tf.Variable(0)
LEARNING_RATE = tf.train.exponential_decay(0.1, global_step, 1, 0.96, staircase=True)

x = tf.Variable(tf.constant(5, dtype=tf.float32), name="x")
y = tf.square(x)
train_op = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(y, global_step=global_step)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(TRAINING_STEPS):
        sess.run(train_op)
        if i % 10 == 0:
            LEARNING_RATE_value = sess.run(LEARNING_RATE)
            x_value = sess.run(x)
            print ("After %s iteration(s): x%s is %f, learning rate is %f."% (i+1, i+1, x_value, LEARNING_RATE_value))

After 1 iteration(s): x1 is 4.000000, learning rate is 0.096000.
After 11 iteration(s): x11 is 0.690561, learning rate is 0.063824.
After 21 iteration(s): x21 is 0.222583, learning rate is 0.042432.
After 31 iteration(s): x31 is 0.106405, learning rate is 0.028210.
After 41 iteration(s): x41 is 0.065548, learning rate is 0.018755.
After 51 iteration(s): x51 is 0.047625, learning rate is 0.012469.
After 61 iteration(s): x61 is 0.038558, learning rate is 0.008290.
After 71 iteration(s): x71 is 0.033523, learning rate is 0.005511.
After 81 iteration(s): x81 is 0.030553, learning rate is 0.003664.
After 91 iteration(s): x91 is 0.028727, learning rate is 0.002436.


# 过拟合
cost Function加上L1 正则化或L2正则化  
其中L1正则化可以将某些参数变成0，从而使参数系数。  
L2不会使参数系数  
(原因参见lasso和ridge regression)


In [ ]:
loss=tf.reduce_mean(tf.square(y_-y))+tf.contrib.layers.l2_regularizer(alpha)(w)

# 滑动平均

#### 1. 定义变量及滑动平均类

In [3]:
import tensorflow as tf
v1 = tf.Variable(0, dtype=tf.float32)
step = tf.Variable(0, trainable=False)
ema = tf.train.ExponentialMovingAverage(0.99, step)
maintain_averages_op = ema.apply([v1]) 

#### 2. 查看不同迭代中变量取值的变化。
$\min (decay,\dfrac{1+num_{updates}}{10+num_{updates}})$
一般情况下，decay设定为一个非常接近1的数

In [4]:
with tf.Session() as sess:
    
    # 初始化
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print (sess.run([v1, ema.average(v1)]))
    
    # 更新变量v1的取值
    sess.run(tf.assign(v1, 5))
    sess.run(maintain_averages_op)#decay=0.1
    print (sess.run([v1, ema.average(v1)])) #0.1*0+0.9*5=4.5
    
    # 更新step和v1的取值
    sess.run(tf.assign(step, 10000))  
    sess.run(tf.assign(v1, 10))
    sess.run(maintain_averages_op)#decay=0.99
    print (sess.run([v1, ema.average(v1)]) )      
    
    # 更新一次v1的滑动平均值
    sess.run(maintain_averages_op)
    print (sess.run([v1, ema.average(v1)])) 


[0.0, 0.0]
[5.0, 4.5]
[10.0, 4.5549998]
[10.0, 4.6094499]


# 正则化

- 加入collection

In [ ]:
weight_loss=tf.multiply(tf.nn.l2_loss(var),wl,name='weight_loss')#wl是L2_loss的系数
tf.add_to_collection('losses',weight_loss)#加入一个collection

- 正则化的和

In [ ]:
tf.add_n(tf.get_collection('losses'),name='total_loss')

把entropy也加入'losses'使代码简洁一些

In [ ]:
tf.add_to_collection('losses',cross_entropy)
tf.add_n(tf.get_collection('losses'),name='total_loss')